In [ ]:
pip install pypdf pdfplumber pytesseract openai llama-index langchain transformers torch coqui-tts elevenlabs gtts pydub ffmpeg-python streamlit soundfile


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 28.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 29.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import files

uploaded = files.upload()  # Prompts you to upload a file
for filename in uploaded.keys():
    print(f"✅ Uploaded file: {filename}")


Saving Python notes.pdf to Python notes.pdf
✅ Uploaded file: Python notes.pdf


In [ ]:
!pip install streamlit pdfplumber pytesseract torch transformers gtts pydub
!apt-get install -y tesseract-ocr
!npm install -g localtunnel
!wget -qO cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x cloudflared


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
⠙⠹⠸⠼⠴⠦⠧
changed 22 packages in 970ms
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [ ]:
%%writefile app.py
import os
import pdfplumber
import pytesseract
from PIL import Image
import streamlit as st
import torch
from transformers import pipeline
from gtts import gTTS
from pydub import AudioSegment

# Set up Streamlit UI
st.title("📄 PDF to Podcast Converter 🎙️")
st.write("Upload a research paper or any PDF, and this app will generate an audio podcast.")

# File upload
uploaded_file = st.file_uploader("📂 Upload a PDF file", type=["pdf"])

# Load Hugging Face summarization model
device = "cuda" if torch.cuda.is_available() else "cpu"
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", truncation=True, device=0 if device == "cuda" else -1)

def extract_text_from_pdf(pdf_file):
    """Extract text from PDF using pdfplumber and OCR for images."""
    text = ""
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            extracted_text = page.extract_text()
            if extracted_text:
                text += extracted_text + "\n"
            else:
                # OCR for image-based PDFs
                image = page.to_image()
                img_path = "temp_page.png"
                image.save(img_path, format="PNG")
                ocr_text = pytesseract.image_to_string(Image.open(img_path))
                text += ocr_text + "\n"

    if not text.strip():
        return "❌ No extractable text found. The PDF might be empty or contain unsupported formats."

    return text.strip()

def chunk_text(text, max_tokens=1024):
    """Split text into smaller chunks to fit model limits."""
    words = text.split()
    chunks = [" ".join(words[i:i+max_tokens]) for i in range(0, len(words), max_tokens)]
    return chunks

def summarize_text(text):
    """Summarize extracted text while handling errors."""
    try:
        text_chunks = chunk_text(text)
        summaries = []
        for chunk in text_chunks:
            if len(chunk.split()) > 50:  # Ensure chunk has enough words for summarization
                summaries.append(summarizer(chunk, max_length=512, min_length=100, do_sample=False)[0]['summary_text'])
            else:
                summaries.append(chunk)  # Keep short chunks as-is
        return " ".join(summaries)
    except Exception as e:
        return f"❌ Error during summarization: {str(e)}"

def text_to_speech(text, output_audio):
    """Convert text to speech and save as MP3."""
    tts = gTTS(text=text, lang="en")
    tts.save(output_audio)

def process_pdf(uploaded_file):
    """Process the uploaded PDF."""
    if uploaded_file is not None:
        temp_pdf_path = "temp_uploaded.pdf"
        with open(temp_pdf_path, "wb") as f:
            f.write(uploaded_file.read())

        st.write("🔍 Extracting text from PDF...")
        text = extract_text_from_pdf(temp_pdf_path)

        if "❌" in text:
            st.error(text)
            return

        st.write("✍️ Summarizing text...")
        summary = summarize_text(text)
        st.text_area("📌 Summary:", summary, height=400)

        if "❌" in summary:
            st.error(summary)
            return

        st.write("🔊 Converting to speech...")
        output_audio = "podcast.mp3"
        text_to_speech(summary, output_audio)

        st.audio(output_audio, format="audio/mp3", start_time=0)
        st.success("✅ Podcast generated successfully!")

if uploaded_file:
    process_pdf(uploaded_file)


Overwriting app.py


In [ ]:
!streamlit run app.py &>/dev/null &
!./cloudflared tunnel --url http://localhost:8501 --no-autoupdate


2025-03-13T11:32:15Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-03-13T11:32:15Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-03-13T11:32:19Z INF +--------------------------------------------------------------------------------------------+
2025-03-13T11:32:19Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-03-13T11:32:19Z INF |  https://provisions-album-surrounding-ak.trycloudflare